In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

class ThreeLayerNN(tf.Module):
    def __init__(self, input_size=3, hidden_size=64, output_size=1):
        super().__init__()
        self.W1 = tf.Variable(tf.random.normal([input_size, hidden_size], stddev=0.01))
        self.b1 = tf.Variable(tf.zeros([hidden_size]))
        self.W2 = tf.Variable(tf.random.normal([hidden_size, hidden_size], stddev=0.01))
        self.b2 = tf.Variable(tf.zeros([hidden_size]))
        self.W3 = tf.Variable(tf.random.normal([hidden_size, output_size], stddev=0.01))
        self.b3 = tf.Variable(tf.zeros([output_size]))

    def __call__(self, X):
        Z1 = tf.einsum('ij,jk->ik', X, self.W1) + self.b1
        A1 = tf.nn.relu(Z1)
        Z2 = tf.einsum('ij,jk->ik', A1, self.W2) + self.b2
        A2 = tf.nn.relu(Z2)
        Z3 = tf.einsum('ij,jk->ik', A2, self.W3) + self.b3
        return Z3


def generate_data(num_samples=1000):
    np.random.seed(42)
    x = np.random.uniform(-2, 2, num_samples)
    y = np.random.uniform(-2, 2, num_samples)
    z = np.random.uniform(-2, 2, num_samples)
    f_xyz = np.sin(x) + np.cos(y) * np.power(z, 2)
    features = np.vstack((x, y, z)).T
    outputs = f_xyz
    return features, outputs

def compute_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

# Generate and prepare data
features, outputs = generate_data()
X_train, X_test, y_train, y_test = train_test_split(features, outputs, test_size=0.2, random_state=42)
X_train_tf = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tf = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_test_tf = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test_tf = tf.convert_to_tensor(y_test, dtype=tf.float32)

# Initialize model and optimizer
model = ThreeLayerNN()
optimizer = tf.optimizers.Adam(learning_rate=0.001)

# Define training step
@tf.function
def train_step(X, y, model, optimizer):
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = compute_loss(y, y_pred)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Training loop
def train(model, optimizer, X_train_tf, y_train_tf, epochs=100, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((X_train_tf, y_train_tf)).shuffle(buffer_size=1024).batch(batch_size)
    for epoch in range(epochs):
        epoch_loss_avg = tf.keras.metrics.Mean()
        for X_batch, y_batch in dataset:
            loss = train_step(X_batch, y_batch, model, optimizer)
            epoch_loss_avg.update_state(loss)
        print(f"Epoch {epoch + 1}: Loss = {epoch_loss_avg.result().numpy()}")

# Train the model
train(model, optimizer, X_train_tf, y_train_tf)

# Evaluate the model
y_pred = model(X_test_tf)
test_loss = compute_loss(y_test_tf, y_pred)
print(f"Test Loss: {test_loss.numpy()}")


Epoch 1: Loss = 2.000098466873169
Epoch 2: Loss = 1.810802936553955
Epoch 3: Loss = 1.6838812828063965
Epoch 4: Loss = 1.6823647022247314
Epoch 5: Loss = 1.6806391477584839
Epoch 6: Loss = 1.6806840896606445
Epoch 7: Loss = 1.6785045862197876
Epoch 8: Loss = 1.6839649677276611
Epoch 9: Loss = 1.6810803413391113
Epoch 10: Loss = 1.680355191230774
Epoch 11: Loss = 1.6780260801315308
Epoch 12: Loss = 1.680052638053894
Epoch 13: Loss = 1.6775925159454346
Epoch 14: Loss = 1.6790844202041626
Epoch 15: Loss = 1.684868335723877
Epoch 16: Loss = 1.679939866065979
Epoch 17: Loss = 1.6825802326202393
Epoch 18: Loss = 1.6765129566192627
Epoch 19: Loss = 1.6792603731155396
Epoch 20: Loss = 1.6859278678894043
Epoch 21: Loss = 1.6778944730758667
Epoch 22: Loss = 1.6779855489730835
Epoch 23: Loss = 1.6791094541549683
Epoch 24: Loss = 1.6802124977111816
Epoch 25: Loss = 1.6766886711120605
Epoch 26: Loss = 1.6782385110855103
Epoch 27: Loss = 1.677445650100708
Epoch 28: Loss = 1.6780738830566406
Epoch 29